# KRAS Mutation Trans Effect on Transcriptomics

This notebook analyzes the trans effect of KRAS mutation on interacting and other proteins Transcriptomics, in Endometrial and Colon cancer.

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import collections
import re
import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import cptac.algorithms as al

en = cptac.Endometrial()
co = cptac.Colon()

Checking that index is up-to-date...Loading acetylproteomics data...Loading clinical data...Loading CNA data...Loading definitions data...Loading miRNA data...Loading phosphoproteomics_gene data...Loading phosphoproteomics_site data...Loading proteomics data...Loading somatic_binary data...Loading somatic data...Loading transcriptomics_circular data...Loading transcriptomics_linear data...Formatting dataframes...Checking that index is up-to-date...Loading clinical data...Loading miRNA data...Loading mutation_binary data...Loading mutation data...Loading phosphoproteomics_normal data...Loading phosphoproteomics_tumor data...Loading proteomics_normal data...Loading proteomics_tumor data...Loading transcriptomics data...Formatting dataframes...

### Investigate KRAS gene

In [2]:
gene = "KRAS"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

# Interacting Proteins: Transcriptomics

## Endometrial

### Generate interacting protein list

In [4]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = en.get_proteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
RAF1
MAPK1
KRAS
MAP2K2
MAPK3
PIK3CA
ERBB3
ERBB2
EGFR
BRAF
PIK3CB
ARAF
MAP2K1
PTPN11
NF1
PIK3CG
NRAS
SRC
GRB2
SOS1
HRAS
CDKN2A
RAP1GDS1
HNRNPC
RASSF2
RASSF5


### Test for significant comparisons in any of interacting proteins

In [5]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = en.append_mutations_to_omics(mutation_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons

No significant comparisons.


## Colon

### Generate interacting protein list

In [10]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = co.get_proteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
RAF1
MAPK1
KRAS
MAP2K2
MAPK3
PIK3CA
ERBB3
ERBB2
EGFR
BRAF
PIK3CB
ARAF
MAP2K1
PTPN11
NF1
NRAS
SRC
GRB2
SOS1
HRAS
CDKN2A
RAP1GDS1
HNRNPC
RASSF2
RASSF5


### Test for significant comparisons in any of interacting proteins

In [11]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = co.append_mutations_to_omics(mutation_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons

No significant comparisons.


# All Proteins: Transcriptomics

## Endometrial

In [14]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = en.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.append_mutations_to_omics(mutation_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  KRAS
MT-ATP6 did not match any columns in transcriptomics dataframe. MT-ATP6_transcriptomics column inserted, but filled with NaN.
MT-ATP8 did not match any columns in transcriptomics dataframe. MT-ATP8_transcriptomics column inserted, but filled with NaN.
MT-CO1 did not match any columns in transcriptomics dataframe. MT-CO1_transcriptomics column inserted, but filled with NaN.
MT-CO2 did not match any columns in transcriptomics dataframe. MT-CO2_transcriptomics column inserted, but filled with NaN.
MT-CO3 did not match any columns in transcriptomics dataframe. MT-CO3_transcriptomics column inserted, but filled with NaN.
MT-CYB did not match any columns in transcriptomics dataframe. MT-CYB_transcriptomics column inserted, but filled with NaN.
MT-ND1 did not match any columns in transcriptomics dataframe. MT-ND1_transcriptomics column inserted, but filled with NaN.
MT-ND2 did not match any columns in transcriptomics dataframe. MT-ND2_transcriptomics column inserted, but filled w

## Colon

In [ ]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = co.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = co.append_mutations_to_omics(mutation_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  KRAS
A1BG did not match any columns in transcriptomics dataframe. A1BG_transcriptomics column inserted, but filled with NaN.
ABCA8 did not match any columns in transcriptomics dataframe. ABCA8_transcriptomics column inserted, but filled with NaN.
ACAN did not match any columns in transcriptomics dataframe. ACAN_transcriptomics column inserted, but filled with NaN.
ACTA1 did not match any columns in transcriptomics dataframe. ACTA1_transcriptomics column inserted, but filled with NaN.
ACTBL2 did not match any columns in transcriptomics dataframe. ACTBL2_transcriptomics column inserted, but filled with NaN.
ACTL8 did not match any columns in transcriptomics dataframe. ACTL8_transcriptomics column inserted, but filled with NaN.
ACTN2 did not match any columns in transcriptomics dataframe. ACTN2_transcriptomics column inserted, but filled with NaN.
ACTN3 did not match any columns in transcriptomics dataframe. ACTN3_transcriptomics column inserted, but filled with NaN.
ADAMTS16 did 

CA7 did not match any columns in transcriptomics dataframe. CA7_transcriptomics column inserted, but filled with NaN.
CADM2 did not match any columns in transcriptomics dataframe. CADM2_transcriptomics column inserted, but filled with NaN.
CADM3 did not match any columns in transcriptomics dataframe. CADM3_transcriptomics column inserted, but filled with NaN.
CALB1 did not match any columns in transcriptomics dataframe. CALB1_transcriptomics column inserted, but filled with NaN.
CALML3 did not match any columns in transcriptomics dataframe. CALML3_transcriptomics column inserted, but filled with NaN.
CALML5 did not match any columns in transcriptomics dataframe. CALML5_transcriptomics column inserted, but filled with NaN.
CALR3 did not match any columns in transcriptomics dataframe. CALR3_transcriptomics column inserted, but filled with NaN.
CAMK2B did not match any columns in transcriptomics dataframe. CAMK2B_transcriptomics column inserted, but filled with NaN.
CAMK4 did not match an

CTNNA3 did not match any columns in transcriptomics dataframe. CTNNA3_transcriptomics column inserted, but filled with NaN.
CTRB2 did not match any columns in transcriptomics dataframe. CTRB2_transcriptomics column inserted, but filled with NaN.
CTRC did not match any columns in transcriptomics dataframe. CTRC_transcriptomics column inserted, but filled with NaN.
CYLC2 did not match any columns in transcriptomics dataframe. CYLC2_transcriptomics column inserted, but filled with NaN.
CYP3A4 did not match any columns in transcriptomics dataframe. CYP3A4_transcriptomics column inserted, but filled with NaN.
CYP4F8 did not match any columns in transcriptomics dataframe. CYP4F8_transcriptomics column inserted, but filled with NaN.
CYS1 did not match any columns in transcriptomics dataframe. CYS1_transcriptomics column inserted, but filled with NaN.
DAPL1 did not match any columns in transcriptomics dataframe. DAPL1_transcriptomics column inserted, but filled with NaN.
DCLK1 did not match an

GNG7 did not match any columns in transcriptomics dataframe. GNG7_transcriptomics column inserted, but filled with NaN.
GNRH2 did not match any columns in transcriptomics dataframe. GNRH2_transcriptomics column inserted, but filled with NaN.
GP1BB did not match any columns in transcriptomics dataframe. GP1BB_transcriptomics column inserted, but filled with NaN.
GP2 did not match any columns in transcriptomics dataframe. GP2_transcriptomics column inserted, but filled with NaN.
GP9 did not match any columns in transcriptomics dataframe. GP9_transcriptomics column inserted, but filled with NaN.
GPX6 did not match any columns in transcriptomics dataframe. GPX6_transcriptomics column inserted, but filled with NaN.
GREM2 did not match any columns in transcriptomics dataframe. GREM2_transcriptomics column inserted, but filled with NaN.
GSDMA did not match any columns in transcriptomics dataframe. GSDMA_transcriptomics column inserted, but filled with NaN.
GSTT2 did not match any columns in t

KRT14 did not match any columns in transcriptomics dataframe. KRT14_transcriptomics column inserted, but filled with NaN.
KRT16 did not match any columns in transcriptomics dataframe. KRT16_transcriptomics column inserted, but filled with NaN.
KRT2 did not match any columns in transcriptomics dataframe. KRT2_transcriptomics column inserted, but filled with NaN.
KRT3 did not match any columns in transcriptomics dataframe. KRT3_transcriptomics column inserted, but filled with NaN.
KRT31 did not match any columns in transcriptomics dataframe. KRT31_transcriptomics column inserted, but filled with NaN.
KRT34 did not match any columns in transcriptomics dataframe. KRT34_transcriptomics column inserted, but filled with NaN.
KRT36 did not match any columns in transcriptomics dataframe. KRT36_transcriptomics column inserted, but filled with NaN.
KRT5 did not match any columns in transcriptomics dataframe. KRT5_transcriptomics column inserted, but filled with NaN.
KRT6A did not match any column

NOS1 did not match any columns in transcriptomics dataframe. NOS1_transcriptomics column inserted, but filled with NaN.
NOS1AP did not match any columns in transcriptomics dataframe. NOS1AP_transcriptomics column inserted, but filled with NaN.
NOVA2 did not match any columns in transcriptomics dataframe. NOVA2_transcriptomics column inserted, but filled with NaN.
NPY did not match any columns in transcriptomics dataframe. NPY_transcriptomics column inserted, but filled with NaN.
NTN1 did not match any columns in transcriptomics dataframe. NTN1_transcriptomics column inserted, but filled with NaN.
OGDHL did not match any columns in transcriptomics dataframe. OGDHL_transcriptomics column inserted, but filled with NaN.
ORM1 did not match any columns in transcriptomics dataframe. ORM1_transcriptomics column inserted, but filled with NaN.
ORM2 did not match any columns in transcriptomics dataframe. ORM2_transcriptomics column inserted, but filled with NaN.
OTC did not match any columns in t

RPL3L did not match any columns in transcriptomics dataframe. RPL3L_transcriptomics column inserted, but filled with NaN.
RPS4Y1 did not match any columns in transcriptomics dataframe. RPS4Y1_transcriptomics column inserted, but filled with NaN.
RPS4Y2 did not match any columns in transcriptomics dataframe. RPS4Y2_transcriptomics column inserted, but filled with NaN.
RTN1 did not match any columns in transcriptomics dataframe. RTN1_transcriptomics column inserted, but filled with NaN.
S100A7 did not match any columns in transcriptomics dataframe. S100A7_transcriptomics column inserted, but filled with NaN.
SAA2-SAA4 did not match any columns in transcriptomics dataframe. SAA2-SAA4_transcriptomics column inserted, but filled with NaN.
SARDH did not match any columns in transcriptomics dataframe. SARDH_transcriptomics column inserted, but filled with NaN.
SCGN did not match any columns in transcriptomics dataframe. SCGN_transcriptomics column inserted, but filled with NaN.
SCN7A did not 

TMEM189-UBE2V1 did not match any columns in transcriptomics dataframe. TMEM189-UBE2V1_transcriptomics column inserted, but filled with NaN.
TMEM35A did not match any columns in transcriptomics dataframe. TMEM35A_transcriptomics column inserted, but filled with NaN.
TMOD1 did not match any columns in transcriptomics dataframe. TMOD1_transcriptomics column inserted, but filled with NaN.
TMPPE did not match any columns in transcriptomics dataframe. TMPPE_transcriptomics column inserted, but filled with NaN.
TNXB did not match any columns in transcriptomics dataframe. TNXB_transcriptomics column inserted, but filled with NaN.
TPPP did not match any columns in transcriptomics dataframe. TPPP_transcriptomics column inserted, but filled with NaN.
TRIM72 did not match any columns in transcriptomics dataframe. TRIM72_transcriptomics column inserted, but filled with NaN.
TTR did not match any columns in transcriptomics dataframe. TTR_transcriptomics column inserted, but filled with NaN.
TUBA3C d